In [1]:
import tiktoken
import torch
torch.manual_seed(123)
tokenizer = tiktoken.get_encoding(encoding_name="gpt2")

# Input Text/Encoding

In [2]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
  raw_text = f.read()

In [3]:
raw_text[:20]

'I HAD always thought'

In [4]:
tokenizer.decode([tokenizer.eot_token])

'<|endoftext|>'

In [5]:
text = raw_text[:20] + '<|endoftext|>'
encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
inputs = torch.tensor(encoded)
text, inputs

('I HAD always thought<|endoftext|>',
 tensor([   40,   367,  2885,  1464,  1807, 50256]))

# Token Embeddings

In [6]:
vocab_size = tokenizer.n_vocab
output_dim = 3
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [7]:
token_embeddings = token_embedding_layer(inputs)
token_embeddings, token_embeddings.shape

(tensor([[ 1.3966e+00, -9.9491e-01, -1.5822e-03],
         [-1.1659e+00,  1.3834e-01, -9.8013e-01],
         [ 1.0122e+00, -6.5152e-01,  8.6051e-02],
         [ 2.5264e-01,  2.1043e+00, -6.4511e-01],
         [-3.1843e-01, -1.2264e-01, -3.9307e-02],
         [-3.1300e-01,  7.5582e-01, -1.2656e+00]], grad_fn=<EmbeddingBackward0>),
 torch.Size([6, 3]))

# Position Embeddings

In [8]:
max_length = len(inputs)
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

In [9]:
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
pos_embeddings, pos_embeddings.shape

(tensor([[ 0.1583,  1.0919,  0.8055],
         [ 1.1262, -0.8857, -1.6315],
         [-0.2570,  0.7127, -0.3622],
         [ 0.9609, -1.3697,  0.1381],
         [-1.2365,  1.9319,  0.4730],
         [ 0.7365,  0.1316,  0.2379]], grad_fn=<EmbeddingBackward0>),
 torch.Size([6, 3]))

# Add them up!

In [10]:
input_embeddings = token_embeddings + pos_embeddings
input_embeddings, input_embeddings.shape

(tensor([[ 1.5549,  0.0969,  0.8039],
         [-0.0397, -0.7474, -2.6117],
         [ 0.7552,  0.0611, -0.2762],
         [ 1.2135,  0.7346, -0.5071],
         [-1.5549,  1.8092,  0.4337],
         [ 0.4235,  0.8874, -1.0277]], grad_fn=<AddBackward0>),
 torch.Size([6, 3]))

# Simple self-attention (no trainable weights)

In [11]:
input_embeddings

tensor([[ 1.5549,  0.0969,  0.8039],
        [-0.0397, -0.7474, -2.6117],
        [ 0.7552,  0.0611, -0.2762],
        [ 1.2135,  0.7346, -0.5071],
        [-1.5549,  1.8092,  0.4337],
        [ 0.4235,  0.8874, -1.0277]], grad_fn=<AddBackward0>)

In [12]:
query = input_embeddings[1] # Second input token will be our query
query

tensor([-0.0397, -0.7474, -2.6117], grad_fn=<SelectBackward0>)

In [13]:
input_embeddings.shape

torch.Size([6, 3])

In [14]:
attn_scores_2 = torch.empty(input_embeddings.shape[0])
attn_scores_2

tensor([0., 0., 0., 0., 0., 0.])

In [15]:
query

tensor([-0.0397, -0.7474, -2.6117], grad_fn=<SelectBackward0>)

In [16]:
for i, x_i in enumerate(input_embeddings):
  dp = torch.dot(x_i, query)
  attn_scores_2[i] = dp
  print(f"dot product of {x_i.round(decimals=2).data} and {query.round(decimals=2).data}:")
  print(" + ".join([f"{x_i[idx]:.2f} * {element:.2f}" for idx, element in enumerate(query)]), end=f" = {dp:.2f}")
  print("\n\n")
attn_scores_2.data

dot product of tensor([1.5500, 0.1000, 0.8000]) and tensor([-0.0400, -0.7500, -2.6100]):
1.55 * -0.04 + 0.10 * -0.75 + 0.80 * -2.61 = -2.23


dot product of tensor([-0.0400, -0.7500, -2.6100]) and tensor([-0.0400, -0.7500, -2.6100]):
-0.04 * -0.04 + -0.75 * -0.75 + -2.61 * -2.61 = 7.38


dot product of tensor([ 0.7600,  0.0600, -0.2800]) and tensor([-0.0400, -0.7500, -2.6100]):
0.76 * -0.04 + 0.06 * -0.75 + -0.28 * -2.61 = 0.65


dot product of tensor([ 1.2100,  0.7300, -0.5100]) and tensor([-0.0400, -0.7500, -2.6100]):
1.21 * -0.04 + 0.73 * -0.75 + -0.51 * -2.61 = 0.73


dot product of tensor([-1.5500,  1.8100,  0.4300]) and tensor([-0.0400, -0.7500, -2.6100]):
-1.55 * -0.04 + 1.81 * -0.75 + 0.43 * -2.61 = -2.42


dot product of tensor([ 0.4200,  0.8900, -1.0300]) and tensor([-0.0400, -0.7500, -2.6100]):
0.42 * -0.04 + 0.89 * -0.75 + -1.03 * -2.61 = 2.00




tensor([-2.2338,  7.3810,  0.6456,  0.7270, -2.4230,  2.0040])

In [17]:
attn_scores = input_embeddings @ input_embeddings.T
mask = attn_scores == attn_scores.max(dim=0, keepdim=True).values
print(mask)

attn_scores = attn_scores * mask
print(attn_scores)

tensor([[ True, False, False, False, False, False],
        [False,  True, False, False, False, False],
        [False, False, False, False, False, False],
        [False, False,  True,  True, False, False],
        [False, False, False, False,  True, False],
        [False, False, False, False, False,  True]])
tensor([[3.0734, -0.0000, 0.0000, 0.0000, -0.0000, -0.0000],
        [-0.0000, 7.3810, 0.0000, 0.0000, -0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, -0.0000, 0.0000],
        [0.0000, 0.0000, 1.1014, 2.2694, -0.0000, 0.0000],
        [-0.0000, -0.0000, -0.0000, -0.0000, 5.8791, 0.0000],
        [-0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 2.0230]],
       grad_fn=<MulBackward0>)
